TextRank

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

In [ ]:
import numpy as np
import pandas as pd

import json
from pandas.io.json import json_normalize

import re
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

DIR = "/content/drive/MyDrive/DACON_LG/data/"
TRAIN_SOURCE = os.path.join(DIR, "train.json")
TEST_SOURCE = os.path.join(DIR, "test.json")

In [ ]:
with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())
    
with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

In [ ]:
train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
uid = 1000
for data in TRAIN_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        train.loc[uid, 'uid'] = uid
        train.loc[uid, 'title'] = data['title']
        train.loc[uid, 'region'] = data['region']
        train.loc[uid, 'context'] = context[:-1]
        train.loc[uid, 'summary'] = data['label'][agenda]['summary']
        uid += 1

test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
uid = 2000
for data in TEST_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        test.loc[uid, 'uid'] = uid
        test.loc[uid, 'title'] = data['title']
        test.loc[uid, 'region'] = data['region']
        test.loc[uid, 'context'] = context[:-1]
        uid += 1

In [ ]:
train['total'] = train.title + ' ' + train.region + ' ' + train.context
test['total'] = test.title + ' ' + test.region + ' ' + test.context

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
!pip install git+https://github.com/lovit/textrank.git

In [ ]:
from konlpy.tag import Komoran
from textrank import KeysentenceSummarizer
import textrank

komoran = Komoran()

def komoran_tokenizer(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenizer,
    min_sim = 0.3,
    verbose = False
)

In [ ]:
texts = test['total'].tolist()

In [ ]:
real_text = []

for i in range(len(texts)):
  real_text.append([[texts[i]]])

In [ ]:
article_list = pd.DataFrame(real_text, columns=['articles'])

In [ ]:
article_list

In [ ]:
real_summary = []

for text in tqdm(article_list['articles']):
    real_keysents = summarizer.summarize(text, topk=7)
    real_summary.append(real_keysents)

In [ ]:
real_extractive = []

for n in range(len(real_text)):
    real_result = real_summary[n][0][2]# + '\n' + real_summary[n][1][2] + '\n' + real_summary[n][2][2]
    real_extractive.append(real_result)

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/DACON_LG/data/sample_submission.csv')

In [ ]:
submission

,uid,summary
0,id_2000-AGENDA_1,요약문
1,id_2000-AGENDA_2,요약문
2,id_2000-AGENDA_3,요약문
3,id_2000-AGENDA_4,요약문
4,id_2000-AGENDA_5,요약문
...,...,...
501,id_2096-AGENDA_2,요약문
502,id_2096-AGENDA_3,요약문
503,id_2096-AGENDA_4,요약문
504,id_2096-AGENDA_5,요약문


In [ ]:
submission['summary'] = real_extractive

In [ ]:
submission.to_csv('/content/drive/MyDrive/DACON_LG/data/dacon_improve_textrank.csv', encoding='utf-8', index=False)